# Import Dependencies

In [1]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Types of Spaces

In [25]:
Discrete(3).sample()

2

In [26]:
Box(0, 1, shape=(3, 3)).sample()

array([[0.1472021 , 0.8254482 , 0.01586655],
       [0.2606284 , 0.7919413 , 0.47123948],
       [0.7862849 , 0.39031598, 0.45014578]], dtype=float32)

In [27]:
Tuple((Discrete(3), Box(0, 1, shape=(3,)))).sample()

(1, array([0.7281385 , 0.96977824, 0.9443628 ], dtype=float32))

In [28]:
Dict({'height': Discrete(2), "speed": Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([95.76574], dtype=float32))])

In [29]:
MultiBinary(4).sample()

array([0, 1, 1, 0], dtype=int8)

In [30]:
MultiDiscrete([5,2,2]).sample()

array([2, 0, 1], dtype=int64)

# Building an Environment
- Build an agent to give us the best shower possible
- Randomly fluctuate temperature
- We want temp between 37 and 39 degrees

In [27]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
   
    def step(self, action):
        # Adjust temperature
        self.state += action - 1
        # Decrease shower time
        self.shower_length -= 1
        # Callculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        # Check end time
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        info = {}
        truncated = {}
        return self.state, reward, done, truncated, info
        
    def render(self):
        # Not implementing rendering...
        pass
    
    def reset(self, seed=None):
        super().reset(seed = seed)
        self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
        self.shower_length = 60
        info = {}
        return self.state, info

In [28]:
env = ShowerEnv()

In [4]:
env.observation_space.sample()

array([10.039273], dtype=float32)

In [5]:
env.action_space.sample()

1

# Test Environment

In [19]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-6
Episode:2 Score:-60
Episode:3 Score:-60
Episode:4 Score:-14
Episode:5 Score:18


In [7]:
env.close()

# Train Model

In [29]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [31]:
model.learn(total_timesteps=40000)

Logging to Training\Logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -34.2    |
| time/              |          |
|    fps             | 1123     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -34.5      |
| time/                   |            |
|    fps                  | 709        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01258566 |
|    clip_fraction        | 0.0826     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.1       |
|    explained_variance   | -0.000545  |
|    learning_

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -24.7      |
| time/                   |            |
|    fps                  | 608        |
|    iterations           | 11         |
|    time_elapsed         | 37         |
|    total_timesteps      | 22528      |
| train/                  |            |
|    approx_kl            | 0.01026277 |
|    clip_fraction        | 0.117      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.09      |
|    explained_variance   | -0.00472   |
|    learning_rate        | 0.0003     |
|    loss                 | 37.7       |
|    n_updates            | 120        |
|    policy_gradient_loss | -0.00993   |
|    value_loss           | 77.1       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

# Save Model

In [34]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [35]:
model.save(shower_path)

In [ ]:
del model

In [ ]:
model = PPO.load(shower_path, env)

# Evaluate Model

In [36]:
evaluate_policy(model, env, n_eval_episodes=10)

C:\Users\damjanh\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(59.4, 0.9165151389911681)